This is the large casestudy for the damage dataset. Data that we are reading is 64Gb and device that I have is 32 GB so technically I cannot play around with this data

In [2]:
from dask.distributed import Client, progress
import tables
import numpy as np


In [3]:
import dask.array as da

In [13]:
path="/home/berkay/Projects/pymks/data/ms_copy_data.hdf5"


In [14]:
tps_hdf5_file = tables.open_file(path, mode='r')


In [15]:
ms_data=tps_hdf5_file.root.ms_data

In [16]:
ms_data

/ms_data (EArray(130000, 257651), shuffle, blosc(5)) ''
  atom := Float32Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := (2, 257651)

In [10]:
# Actually it has a chunk shape (from pytables) as well  maybe it could be an issue too

In [11]:
# import dask.dataframe as dd


In [12]:
# dask_data=dd.read_hdf(path,'/*',mode="r")

In [13]:
dask_array=da.from_array(ms_data)

In [14]:
    from dask_ml.decomposition import PCA
    pca=PCA(n_components=50,random_state=4711)

In [15]:
pca.fit(dask_array)

distributed.protocol.core - CRITICAL - Failed to Serialize
Traceback (most recent call last):
  File "/home/berkay/anaconda3/lib/python3.7/site-packages/distributed/protocol/core.py", line 44, in dumps
    for key, value in data.items()
  File "/home/berkay/anaconda3/lib/python3.7/site-packages/distributed/protocol/core.py", line 45, in <dictcomp>
    if type(value) is Serialize
  File "/home/berkay/anaconda3/lib/python3.7/site-packages/distributed/protocol/serialize.py", line 210, in serialize
    raise TypeError(msg, str(x)[:10000])
TypeError: ('Could not serialize object of type EArray.', "/ms_data (EArray(130000, 257651), shuffle, blosc(5)) ''")
distributed.comm.utils - ERROR - ('Could not serialize object of type EArray.', "/ms_data (EArray(130000, 257651), shuffle, blosc(5)) ''")
Traceback (most recent call last):
  File "/home/berkay/anaconda3/lib/python3.7/site-packages/distributed/comm/utils.py", line 33, in _to_frames
    msg, serializers=serializers, on_error=on_error, conte

KeyboardInterrupt: 

In [ ]:
51**3

# Data Construction
Creating the Large Dask Array From HDF file:<br/>
Target Data is n* 51* 51*51<br/>
So I will first read as NP array then will construct the dask array. Basically , we are stitching them together.

In [ ]:
datasave=np.array(ms_data[1:4000,0:51**3])


In [ ]:
import dask.dataframe as dd

In [ ]:
df1=da.from_array(datasave)

In [ ]:
df1.to_hdf("datafty.hdf5",'/data')

In [ ]:
type(df1)

In [ ]:
# I guess I dont need to do that
chunksize=13000
dask_arrays = []
for i in range(10):
    import dask.array as da
    import numpy as np
    import tables
    path="/home/berkay/Projects/pymks/data/ms_copy_data.hdf5"
    tps_hdf5_file = tables.open_file(path, mode='r')
    ms_data=tps_hdf5_file.root.ms_data
#     ms_chunk=np.array(ms_data[i*chunksize:(i+1)*chunksize])
#     ms_chunk=ms_chunk[:,0:51**3]
#     ms_chunk=ms_chunk.reshape((chunksize,51,51,51))
    ms_chunk=da.from_array(np.array(ms_data[i*chunksize:(i+1)*chunksize,0:51**3]).reshape(chunksize,51,51,51),chunks=(chunksize,51,51,51))
    print("%d Chunks completed"%(i+1))
    dask_arrays.append(ms_chunk)
    %reset

In [ ]:
# ms_data.shape[0]//chunksize
x = da.concatenate(dask_arrays, axis=0)  # concatenate arrays along first axis

In [ ]:
x

In [ ]:
type(x)

Save the small chunk of the data with pytables

In [17]:
h5file = tables.open_file("new_sample2.h5", "w", driver="H5FD_CORE")
val=500
a = h5file.create_array(h5file.root, "array",ms_data[0:val,0:51**3])
h5file.close()

HDF5ExtError: HDF5 error back trace

  File "H5F.c", line 444, in H5Fcreate
    unable to create file
  File "H5Fint.c", line 1567, in H5F_open
    unable to lock the file
  File "H5FD.c", line 1640, in H5FD_lock
    driver lock request failed
  File "H5FDcore.c", line 1519, in H5FD_core_lock
    unable to lock file, errno = 11, error message = 'Resource temporarily unavailable'

End of HDF5 error back trace

Unable to open/create file 'new_sample2.h5'

In [12]:
h5file.close()

NameError: name 'h5file' is not defined

# Start from Here
Unpack the data with pytables

In [65]:
import dask.array as da
import numpy as np

In [66]:
import tables
import dask.array as da

h5file = tables.open_file("new_sample2.h5", "r")

In [67]:
data=h5file.root.array

In [68]:
data

/array (Array(1500, 132651)) ''
  atom := Float32Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None

In [69]:
data=da.from_array(data,chunks=(100,132651))


In [70]:
data=data.reshape(val,51,51,51)

In [71]:
# data.rechunk((100,51,51,51))

In [72]:
data

dask.array<reshape, shape=(1500, 51, 51, 51), dtype=float32, chunksize=(100, 51, 51, 51), chunktype=numpy.ndarray>

In [73]:
from pymks.fmks.bases.primitive import PrimitiveTransformer
from pymks.fmks.correlations import TwoPointCorrelation,FlattenTransformer
from dask_ml.decomposition import PCA
from sklearn.pipeline import Pipeline

In [74]:
# bl = Pipeline(steps=[
#     ("discritize",PrimitiveTransformer(n_state=2, min_=0.0, max_=1.0)),
#     ("Correlations",TwoPointCorrelation(periodic_boundary=True, cutoff=15,correlations=[(1,1)])),
#     ('flatten', FlattenTransformer()),  
#     ('reducer',PCA(n_components=3))])

# # There is an issue 

In [75]:
data

dask.array<reshape, shape=(1500, 51, 51, 51), dtype=float32, chunksize=(100, 51, 51, 51), chunktype=numpy.ndarray>

In [76]:
bl = Pipeline(steps=[
    ("discritize",PrimitiveTransformer(n_state=2, min_=0.0, max_=1.0)),
    ("Correlations",TwoPointCorrelation(periodic_boundary=True, cutoff=15,correlations=[(1,1)])),
    ('flatten', FlattenTransformer()),  
    ('reducer',PCA(n_components=3))])

# There is an issue 

In [77]:
bl.fit(data)


Pipeline(memory=None,
         steps=[('discritize',
                 PrimitiveTransformer(chunks=None, max_=1.0, min_=0.0,
                                      n_state=2)),
                ('Correlations',
                 TwoPointCorrelation(correlations=[(1, 1)], cutoff=15,
                                     periodic_boundary=True)),
                ('flatten', FlattenTransformer()),
                ('reducer',
                 PCA(copy=True, iterated_power=0, n_components=3,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False))],
         verbose=False)

In [78]:
bl.transform(data)[0]

dask.array<getitem, shape=(3,), dtype=float64, chunksize=(3,), chunktype=numpy.ndarray>

In [79]:
bl.transform(data).compute()

array([[-2.55485596e-01,  1.10536501e-01,  5.61990366e-02],
       [ 1.94899162e-02,  7.04576829e-02, -1.00628913e-01],
       [-4.12979853e-01,  2.21059392e-03, -7.13196159e-02],
       ...,
       [ 8.26975640e-01,  4.47431318e-02,  8.84732479e-02],
       [ 6.99686336e-01, -3.32680875e-02, -2.24011624e-02],
       [-2.42821911e-01,  5.44820204e-03,  3.01266767e-04]])

This does not give a memory issue but for big data it crashes

In [ ]:
for i in range(10):
    print (i)
    %reset